### 再帰を使う例

In [ ]:
import random


def recursive_probe(x,y,num,bombs,mapsize):
    #(x,y)が盤の中にある場合のみ処理する。
    if 0 <= x < mapsize[0] and 0 <= y < mapsize[1]:
        for neix in range(x-1,x+2):         #(x,y)の周囲9点について、
            for neiy in range(y-1,y+2):
                if (neix,neiy) not in num:  #その場所はまだ誰も調べていないなら
                    bombcount = countbombs(neix,neiy,bombs)
                    num[(neix,neiy)] = bombcount
                    if bombcount == 0:
                        recursive_probe(neix,neiy,num,bombs,mapsize)


def showmap(num,mapsize):
    print(" |",end="")
    for posx in range(mapsize[0]):
        print("{0} ".format(posx),end="")
    print()
    print("-+",end="")
    for posx in range(mapsize[0]):
        print("--",end="")
    print()
    for posy in range(mapsize[1]):
        print("{0}|".format(posy),end="")
        for posx in range(mapsize[0]):
            if (posx,posy) in num:
                bombcount = num[(posx,posy)]
                if bombcount == 0:
                    print("  ",end="")
                else:
                    print("{0} ".format(bombcount),end="")
            else:
                print("- ",end="")
        print()

        
def countbombs(x,y,bombs):
    n = 0
    for neix in range(x-1,x+2):
        for neiy in range(y-1,y+2):
            if (neix,neiy) in bombs:
                n += 1
    return n



def makemap(mapsize,nbomb):
    bombs = set()
    for i in range(nbomb):
        while True:                 #無条件ループ
            x = random.randint(0,mapsize[0]-1)
            y = random.randint(0,mapsize[1]-1)
            if (x,y) not in bombs:
                break               #whileループから抜ける
        bombs.add((x,y))
    return bombs

nbomb = 10
mapsize = (10,10)
bombs = makemap(mapsize,nbomb)
num   = dict()

while True:                         #また無条件ループ
    showmap(num,mapsize)
    x,y=eval(input("X Y:"))
    if (x,y) in bombs:
        print("BOMB!")
        break                       #whileループから抜ける
    else:
        bombcount = countbombs(x,y,bombs)
        print("There are {0} bombs near ({1},{2}).".format(bombcount,x,y))
        num[(x,y)] = bombcount
        if bombcount == 0:
            recursive_probe(x,y,num,bombs,mapsize)

### 待ち行列を使う例

In [ ]:
import random


def probe_one(queue,num,bombs,mapsize):
    # 筆頭をとりだす
    x,y = queue.pop(0)
    #(x,y)が盤の中にある場合のみ処理する。
    if 0 <= x mapsize[0] and 0 <= y < mapsize[1]:
        for neix in (x-1,x,x+1):         #(x,y)の周囲9点について、
            for neiy in (y-1,y,y+1):
                if (neix,neiy) not in num:  #その場所はまだ誰も調べていないなら
                    bombcount = countbombs(neix,neiy,bombs)
                    num[(neix,neiy)] = bombcount
                    if bombcount == 0:      #周囲に爆弾がない場合には、その点をqueueに追加する。
                        queue.append((neix,neiy))

                        
def showmap(num,mapsize):
    print(" |",end="")
    for posx in range(mapsize[0]):
        print("{0} ".format(posx),end="")
    print()
    print("-+",end="")
    for posx in range(mapsize[0]):
        print("--",end="")
    print()
    for posy in range(mapsize[1]):
        print("{0}|".format(posy),end="")
        for posx in range(mapsize[0]):
            if (posx,posy) in num:
                bombcount = num[(posx,posy)]
                if bombcount == 0:
                    print("  ",end="")
                else:
                    print("{0} ".format(bombcount),end="")
            else:
                print("- ",end="")
        print()

        
def countbombs(x,y,bombs):
    n = 0
    for neix in range(x-1,x+2):
        for neiy in range(y-1,y+2):
            if (neix,neiy) in bombs:
                n += 1
    return n



def makemap(mapsize,nbomb):
    bombs = set()
    for i in range(nbomb):
        while True:                 #無条件ループ
            x = random.randint(0,mapsize[0]-1)
            y = random.randint(0,mapsize[1]-1)
            if (x,y) not in bombs:
                break               #whileループから抜ける
        bombs.add((x,y))
    return bombs

nbomb = 10
mapsize = (10,10)
bombs = makemap(mapsize,nbomb)
num   = dict()

while True:                         #また無条件ループ
    showmap(num,mapsize)
    x,y=eval(input("X Y:"))
    if (x,y) in bombs:
        print("BOMB!")
        break                       #whileループから抜ける
    else:
        bombcount = countbombs(x,y,bombs)
        print("There are {0} bombs near ({1},{2}).".format(bombcount,x,y))
        num[(x,y)] = bombcount
        if bombcount == 0:
            queue = [(x,y)]
            while len(queue) > 0:
                print(queue)
                probe_one(queue,num,bombs,mapsize)

## プロ編
さらに、先読みアシスト機能を付けてみる。

確実に爆弾がある場所がわかったら、その周囲の点に関し、ほかの爆弾の位置も確定できるかどうかを見る。

まず、爆弾の場所をおさめる集合を作る。bombs集合は、プレーヤー側から見えてはいけないので、それとは別にfoundという集合を作る。

In [ ]:
found = set()

ある点の周囲にある爆弾の個数がn個で、まだ開いていないパネルもn個なら、開いていないパネルすべてに爆弾があることになる。まずはこのケースだけを実装しよう。

この処理は、人間が指定した座標とは関係なく、毎回すべての(爆弾数が判明している=numに含まれている)格子点で行う。この処理をassist1()としよう。(処理すべき格子点は減らせる気がするが、とりあえず確実に動くことを優先する)

In [ ]:
# まだprobeしていない格子の個数を数える。やりかたはcountbombsと同じ。
# ただし、mapの外は数えない。個数ではなく位置のリストを返す。
def umprobed(x,y,num,mapsize):
    locs = set()
    for neix in range(x-1,x+2):
        if neix in range(mapsize[0]):
            for neiy in range(y-1,y+2):
                if neiy in range(mapsize[1]) :
                    if (neix,neiy) not in num:
                        locs.add((neix,neiy))
    return locs



# 爆弾が確定した格子の個数を数える。やりかたはcountbombsと同じ。
# 個数ではなく位置のリストを返す。
def already_found(x,y,found):
    locs = set()
    for neix in range(x-1,x+2):
        for neiy in range(y-1,y+2):
            if (neix,neiy) in found:
                locs.add((neix,neiy))
    return locs



def assist1(num,bombs,mapsize,found):
    for rep in range(5):            #5回確認する。冗長。        
        queue = list(num)           #辞書numのキーだけのリストを作る。
        print(queue)
        while len(queue) > 0:
            pos = queue.pop(0)      #先頭要素をとりだす。
            x,y = pos
            nb  = countbombs(x,y,bombs)     #xyの周囲の爆弾の総数
            unk = umprobed(x,y,num,mapsize) #xyの周囲のまだprobeしていない場所
            fnd = already_found(x,y,found)  #xyの周囲で爆弾確定の場所
            print("Checking ({0},{1}): bombs{2} unk{3} fnd{4}".format(x,y,nb,len(unk),len(fnd)))
            if nb == len(unk):
                #すべて爆弾だ!
                #爆弾場所集合に追加する
                print("Bombs are there:",unk)
                found |= unk
            elif nb == len(fnd):
                #周囲のすべての爆弾はもう場所がわかっている。
                #残るunknownな場所には爆弾はない。
                print("No bombs are there:",unk-fnd)
                for p in (unk - fnd):
                    xx,yy = p
                    bombcount = countbombs(xx,yy,bombs)
                    num[(xx,yy)] = bombcount
                    queue.append(p)
                    if bombcount == 0:
                        recursive_probe(xx,yy,num,bombs,mapsize)
                showmap(num,mapsize,found)



            
#以下はほとんど同じ。

import random


def recursive_probe(x,y,num,bombs,mapsize):
    #(x,y)が盤の中にある場合のみ処理する。
    if x in range(mapsize[0]) and y in range(mapsize[1]):
        for neix in range(x-1,x+2):         #(x,y)の周囲9点について、
            if neix in range(mapsize[0]):
                for neiy in range(y-1,y+2):
                    if neiy in range(mapsize[1]):
                        if (neix,neiy) not in num:  #その場所はまだ誰も調べていないなら
                            bombcount = countbombs(neix,neiy,bombs)
                            num[(neix,neiy)] = bombcount
                            if bombcount == 0:
                                recursive_probe(neix,neiy,num,bombs,mapsize)


def showmap(num,mapsize,found):
    print(" |",end="")
    for posx in range(mapsize[0]):
        print("{0} ".format(posx),end="")
    print()
    print("-+",end="")
    for posx in range(mapsize[0]):
        print("--",end="")
    print()
    for posy in range(mapsize[1]):
        print("{0}|".format(posy),end="")
        for posx in range(mapsize[0]):
            if (posx,posy) in num:
                bombcount = num[(posx,posy)]
                if bombcount == 0:
                    print("  ",end="")
                else:
                    print("{0} ".format(bombcount),end="")
            else:
                if (posx,posy) in found:
                    print("* ",end="")
                else:
                    print("- ",end="")

        print()

        
def countbombs(x,y,bombs):
    n = 0
    for neix in range(x-1,x+2):
        for neiy in range(y-1,y+2):
            if (neix,neiy) in bombs:
                n += 1
    return n



def makemap(mapsize,nbomb):
    bombs = set()
    for i in range(nbomb):
        while True:                 #無条件ループ
            x = random.randint(0,mapsize[0]-1)
            y = random.randint(0,mapsize[1]-1)
            if (x,y) not in bombs:
                break               #whileループから抜ける
        bombs.add((x,y))
    return bombs

nbomb = 10
mapsize = (10,10)
bombs = makemap(mapsize,nbomb)
num   = dict()
found = set()

while True:                         #また無条件ループ
    showmap(num,mapsize,found)
    x,y=eval(input("X Y:"))
    if (x,y) in bombs:
        print("BOMB!")
        break                       #whileループから抜ける
    else:
        bombcount = countbombs(x,y,bombs)
        print("There are {0} bombs near ({1},{2}).".format(bombcount,x,y))
        num[(x,y)] = bombcount
        if bombcount == 0:
            recursive_probe(x,y,num,bombs,mapsize)
    #全部おわったあとでアシスト機能を呼びだす。
    showmap(num,mapsize,found)
    assist1(num,bombs,mapsize,found)

アシスト機能は事実上コンピュータによる求解になってしまった。

1種類の求解アルゴリズムだけで解けてしまうので、マインスイーパーはほかのパズルに比べるとずいぶん単純であることがわかった。